In [1]:
# CMSC426 Project 3
# GROUP MEMBERS: Andrew Sumner (asumner1), Teimuraz Trapaidze (ttrapaid)
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
from scipy import ndimage
import math
import random
import sklearn as sk
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# CONVERT IMAGES TO GRAYSCALE?????

In [2]:
folders = ["images/dolphin", "images/airplanes", "images/Leopards"]

def get_imgs_from_folders(folders):
    # Return to this
    # This must be all images, not just dolphin images, and we must track the label
    image_sets = []
    for folder in folders:
        images = []
        i = 0
        for filename in os.listdir(folder):
            i = i + 1
            image = img.imread(os.path.join(folder,filename))
            images.append(image)
            if i >= 65:
                break
        image_sets.append(images)
    return image_sets

all_images = get_imgs_from_folders(folders)

images = all_images[0] # REMOVE THIS LATER!!!!!!!!!!!!!!!!
#print(images)

# ensure feature imbalance is taken care of

In [3]:
# splitting train and test
# note that we already know the labels since the folders are still
# separated within the output data structure (for now)
train_subsets = []
test_subsets = []
for img_set in all_images:
    train_x, test_x = train_test_split(img_set, test_size=0.1)
    train_subsets.append(train_x)
    test_subsets.append(test_x)
    
#print(len(test_subsets[0]))

In [4]:
# supposedly we get descriptors for all images in a category, then 
# stack ? 
desc_all_imgs = []
kp_all_imgs = []
min_num_kp = 999999999999999

# FIRST LOOP FINDS MIN KP BOUND
for img_set in all_images:
    for image in img_set:
        sift = cv2.SIFT_create()
        kp, descriptors = sift.detectAndCompute(image, None)
        if len(kp) < min_num_kp:
            min_num_kp = len(kp)

print("MIN NUM KP = ", min_num_kp)
# SECOND LOOP FINDS BEST KEYPOINTS WHERE ALL IMGS HAVE SAME (MIN) NUM
for img_set in all_images:
    for image in img_set:
        sift = cv2.SIFT_create(nfeatures=min_num_kp)
        kp, descriptors = sift.detectAndCompute(image, None)
        #TRIM DESCRIPTORS HERE
        print(len(descriptors))
        desc_all_imgs.append(descriptors)
        kp_all_imgs = kp_all_imgs + kp

#     print(descriptors.shape)
#     for pt, desc in zip(kp, descriptors):
#         (x,y) = pt.pt
#         print((x,y), desc)

desc_all_imgs = tuple(desc_all_imgs)
desc_all_imgs = np.vstack(desc_all_imgs)
#print(desc_all_imgs)
print()
print(desc_all_imgs.shape)

MIN NUM KP =  34
34
34
34
34
34
35
35
35
34
34
35
35
34
34
34
34
35
34
34
34
34
34
34
34
34
34
34
35
34
36
34
34
34
34
34
34
36
34
34
34
34
34
34
34
34
34
34
34
34
34
34
35
34
35
35
34
34
34
34
34
34
34
34
35
34
34
34
35
34
34
34
34
34
34
34
34
34
35
34
35
34
34
34
34
34
35
35
34
34
34
35
36
35
34
34
34
34
34
34
34
35
34
34
34
34
34
34
34
34
34
34
34
35
34
34
34
34
35
34
34
34
34
34
34
35
34
34
34
35
34
35
34
34
34
34
34
34
34
35
34
35
34
34
34
34
34
34
34
35
35
35
34
34
35
34
34
34
35
35
34
34
35
34
34
34
35
34
34
34
34
34
34
34
34
34
35
34
34
34
34
34
34
34
34
34
34
35
34
34
35
34
34
34
34
34

(6673, 128)


In [5]:
def clustering(keypoints, descriptors, k):
    num_descs = len(keypoints)
    
    center_descs = []
    # initial random k centers
    for i in range(k):
        center = random.randint(0, num_descs-1)
        center_descs.append(descriptors[center])
        
    check = False
    
    while (not check):
        clusters = {i:[] for i in range(k)}
            
        for descriptor in descriptors:
            closest = 99999999999
            i = 0
            for center_desc in center_descs:
                dist = np.sum((center_desc-descriptor)**2)

#                 dist = np.linalg.norm(center_desc - descriptor)
                if dist < closest:
                    closest = dist
                    closest_center = i
                i += 1

            clusters[closest_center].append(descriptor)
            
        old_centers = center_descs
        center_descs = []
        
        for center_num, cluster in clusters.items():
            if (len(cluster) > 0):
                cluster = np.vstack(cluster)
#                 new_center = np.sum(cluster, axis=0)/np.linalg.norm(cluster)
                new_center = cluster.mean(axis=0)
                center_descs.append(new_center)
        
        check = True
        for old, new in zip(old_centers, center_descs):
            if not np.array_equal(old, new):
                check = False
                break
        
        

    return center_descs

In [30]:
def clustering2(descriptors, k):
    indices = range(0, len(descriptors))
    centroid_indices = random.sample(indices, k)
    
    centroids = []
    for i in centroid_indices:
        centroids.append(descriptors[i])
    old_centroids = []
    
    flag = False
    
    while (not flag):
        centroid_groupings = {centroid_idx:[] for centroid_idx in range(len(centroids))}
        for row in descriptors:
            min_dist = 9999999999999999
            min_centroid = None
            min_centroid_idx = -1
            i = -1
            for centroid in centroids:
                i = i+1
                dist = np.linalg.norm(row - centroid)
                if dist < min_dist:
                    min_dist = dist
                    min_centroid = centroid
                    min_centroid_idx = i
            (centroid_groupings[min_centroid_idx]).append(row)
        
        old_centroids = centroids.copy()
        for i in range(len(centroids)):
            cluster_descs = centroid_groupings[i]
            #print(len(cluster_descs))
            cluster_descs = tuple(cluster_descs)
            cluster_descs = np.vstack(cluster_descs)
            new_centroid = cluster_descs.mean(axis=0)
            #print(new_centroid)
            #print(new_centroid.shape)
            centroids[i] = new_centroid
            #print(centroids)
        #print(centroids)
        #print(old_centroids)
        flag = np.array_equal(
            np.vstack(tuple(centroids)), 
            np.vstack(tuple(old_centroids))
        )
        print(flag)
        print(len(centroids))
    return centroids

In [32]:
# i havent checked this yet, but if you were to sum the number of key 
# points in each cluster, it should equal the number of descriptors 

# this is for one image

# issue: why is the number of clusters decreasing with each iteration

#centers = clustering(kp_all_imgs, desc_all_imgs, 400)

centers = clustering2(desc_all_imgs, 400)

False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
True
400


In [33]:
print(centers)

[array([  8.142858  ,   6.714286  ,  27.        ,  41.142857  ,
        16.285715  ,   1.8571428 ,   3.5714285 ,   8.142858  ,
        99.        ,  39.        ,  16.285715  ,  13.142858  ,
         5.        ,   5.142857  ,   9.428572  ,  73.85714   ,
        92.57143   ,  42.        ,  16.428572  ,  28.714285  ,
        10.285714  ,   6.        ,  27.        ,  74.14286   ,
        20.428572  ,  31.142857  ,  20.714285  ,  41.142857  ,
        25.571428  ,  17.714285  ,  30.285715  ,  11.571428  ,
        27.285715  ,  13.        ,  12.285714  ,  11.857142  ,
         7.857143  ,   3.4285715 ,  11.571428  ,  15.714286  ,
       123.        ,  24.571428  ,   2.857143  ,   4.        ,
         6.285714  ,   9.142858  ,  30.857143  , 114.42857   ,
        55.714287  ,  16.857143  ,  14.857142  ,  50.714287  ,
        86.42857   ,  21.        ,  33.714287  ,  83.85714   ,
        35.857143  ,  30.142857  ,  28.857143  ,  39.142857  ,
        26.714285  ,  13.        ,  29.        ,  40.1

In [ ]:
def generate_histogram(descriptors, centers): # descriptors is for a single image
    hist_arr = []
    num_bins = len(centers)
    for row in descriptors:
        min_dist = 9999999999999999999
        min_center_idx = -1
        center_idx = -1
        for center in centers:
            center_idx = center_idx + 1
            dist = np.linalg.norm(row - center)
            if dist < min_dist:
                min_dist = dist
                min_center_idx = center_idx
        hist_arr.append(min_center_idx)
    hist = np.histogram(hist_arr, bins=num_bins)
    plt.hist(hist_arr, bins=num_bins)
    plt.show()
    return hist

In [ ]:
# svm

In [ ]:
# use this later once we have our test and train cleaned up

svclassifier = sk.svm.SVC(kernel='linear')
svclassifier.fit(x_train, y_train)
y_test_pred = svclassifier.predict(x_test)

print(sk.metrics.classification_report(y_test, y_test_pred))